In [1]:
import random
import time
import torch
from torch import nn
import os
import numpy as np

no_appended = False # True if you want to use the no_appended (high level)
learning_rate = 1e-5
decay = 0.9
decay_term = 9
task = 'object'
model_type = 't5-base'

/home/cz/.local/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


create a new labelling dataset that matches each obj label to the splitted obj

In [2]:
#Set device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

## Load Dataset and preprocessing

In [3]:
import pickle

template_by_label = pickle.load(open('data/alfred_data/alfred_dicts/template_by_label.p', 'rb'))
train_set = pickle.load(open('data/alfred_data/train_text_with_ppdl_low_appended.p', 'rb'))
val_set_seen = pickle.load(open('data/alfred_data/val_seen_text_with_ppdl_low_appended.p', 'rb'))
val_set_unseen = pickle.load(open('data/alfred_data/val_unseen_text_with_ppdl_low_appended.p', 'rb'))

obj2idx = pickle.load(open('data/alfred_data/alfred_dicts/obj2idx.p', 'rb'))
idx2obj = pickle.load(open('data/alfred_data/alfred_dicts/idx2obj.p', 'rb'))
idx2recep = pickle.load(open('data/alfred_data/alfred_dicts/idx2recep.p', 'rb'))
recep2idx = pickle.load(open('data/alfred_data/alfred_dicts/recep2idx.p', 'rb'))
toggle2idx = pickle.load(open('data/alfred_data/alfred_dicts/toggle2idx.p', 'rb'))

label_arg = 'object_targets'; num_labels = len(obj2idx) # 127
idx2obj

{0: 'Alarm Clock',
 1: 'Aluminum Foil',
 2: 'Apple',
 3: 'Apple Sliced',
 4: 'Arm Chair',
 5: 'Baseball Bat',
 6: 'Basket Ball',
 7: 'Bathtub',
 8: 'Bathtub Basin',
 9: 'Bed',
 10: 'Blinds',
 11: 'Book',
 12: 'Boots',
 13: 'Glassbottle',
 14: 'Bowl',
 15: 'Box',
 16: 'Bread',
 17: 'Bread Sliced',
 18: 'Butter Knife',
 19: 'Cabinet',
 20: 'Candle',
 21: 'CD',
 22: 'Cell Phone',
 23: 'Chair',
 24: 'Cloth',
 25: 'Coffee Machine',
 26: 'Coffee Table',
 27: 'Counter Top',
 28: 'Credit Card',
 29: 'Cup',
 30: 'Curtains',
 31: 'Desk',
 32: 'Desk Lamp',
 33: 'Desktop',
 34: 'Dining Table',
 35: 'Dish Sponge',
 36: 'Dog Bed',
 37: 'Drawer',
 38: 'Dresser',
 39: 'Dumbbell',
 40: 'Egg',
 41: 'Egg Cracked',
 42: 'Faucet',
 43: 'Floor',
 44: 'Floor Lamp',
 45: 'Footstool',
 46: 'Fork',
 47: 'Fridge',
 48: 'Garbage Bag',
 49: 'Garbage Can',
 50: 'Hand Towel',
 51: 'Hand Towel Holder',
 52: 'House Plant',
 53: 'Kettle',
 54: 'Key Chain',
 55: 'Knife',
 56: 'Ladle',
 57: 'Laptop',
 58: 'Laundry Hamper

In [4]:
train_prefix_high = train_set['x_prefix']
train_prefix_low = train_set['x_low_prefix']
label_parameters = train_set['parameters']
print(train_prefix_high[88])
print(train_prefix_low[88])
print(label_parameters[88])

summarize the task parameters: move the alarm clock from one desk to another one
summarize the task parameters: move the alarm clock from one desk to another one[SEP]turn right turn left walk towards the small desk on the left side[SEP]pick up the alarm clock that is on the desk[SEP]turn right walk towards the wall turn to face the desk to the left[SEP]put the alarm clock on the desk
task_type: pick_and_place_simple, object_target: Alarm Clock, mrecep_target: None, parent_target: Desk


In [5]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base').to(device)

tokenize and encode the labels for each object

In [6]:
input_ids = tokenizer(train_prefix_high, return_tensors='pt', padding='longest', truncation=True).input_ids.to(device) # tokenized input
labels = tokenizer(label_parameters, return_tensors='pt', padding='longest', truncation=True).input_ids.to(device) # tokenized labels
labels[labels == tokenizer.pad_token_id] = -100

print("input_ids shape", input_ids.shape)
print("labels shape", labels.shape)

input_ids shape torch.Size([21025, 41])
labels shape torch.Size([21025, 48])


do the same for validation labels

In [7]:
val_seen_prefix_high = val_set_seen['x_prefix']
val_seen_prefix_low = val_set_seen['x_low_prefix']
val_seen_label_parameters = val_set_seen['parameters']
print(val_seen_prefix_high[67])
print(val_seen_prefix_low[67])
print(val_seen_label_parameters[67])

summarize the task parameters: place a vase on a coffee table
summarize the task parameters: place a vase on a coffee table[SEP]turn right and cross the room turn right  and walk to the tc stand on your left turn to face the tv stand[SEP]pick up the blue vase that is sitting on the tv stand [SEP]turn around to face the black coffee table in[SEP]place the vase on the coffee table to the left of the computer
task_type: pick_and_place_simple, object_target: Vase, mrecep_target: None, parent_target: Coffee Table


In [8]:
from torch.utils.data import DataLoader, TensorDataset

input_ids_val_seen = tokenizer(val_seen_prefix_high, return_tensors='pt', padding='longest', truncation=True).input_ids.to(device) # tokenized val seen input
labels_val_seen = tokenizer(val_seen_label_parameters, return_tensors='pt', padding='longest', truncation=True).input_ids.to(device) # tokenized labels
labels_val_seen[labels_val_seen == tokenizer.pad_token_id] = -100

val_seen_dataset = TensorDataset(input_ids_val_seen, labels_val_seen)
val_seen_loader = DataLoader(val_seen_dataset, batch_size=100)  # Batch size set to 100

## Training
model directory

In [9]:
print('Model initialized successfully')
model.train()
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=learning_rate)
if 'base' in model_type:
    N= 32
else:
    N = 16

Model initialized successfully


/home/cz/.local/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [10]:
def accuracy(outputs_batch, labels_batch):
    y_hat = outputs_batch.argmax(dim=-1)
    batch_acc = (y_hat == labels_batch).float().mean()
    return batch_acc # returns the accuracy of the batch

# def accurate_total(y_pred, y_batch): #calculates the number of accurate predictions 
#     y_hat = torch.argmax(y_pred, dim=-1)
#     correct_predictions = (y_hat == y_batch).float()

#     # Count the total number of correct predictions
#     num_accurate = correct_predictions.sum().item()

#     return num_accurate

In [11]:
accuracy_dictionary = {'training_loss': [], 'training':[], 'val_seen_loss': [],'val_seen_acc':[]}
start_train_time = time.time()

for t in range(50):
    model.train()
    if t>0 and (t+1)%decay_term ==0:
        learning_rate *= decay
        optimizer = AdamW(model.parameters(), lr=learning_rate)
    avg_training_loss = 0.0
    avg_training_acc = 0.0
    for b in range(int(input_ids.shape[0]/N)):
        print('input_ids shape: ', input_ids.shape)
        input_ids_batch = input_ids[N*b:N*(b+1)].to(device) # has shape [64, 475]
        print('input_ids_batch shape: ', input_ids_batch.shape)
        labels_batch = labels[N*b:N*(b+1)].to(device)
        optimizer.zero_grad() # clear gradients
        print('labels_batch shape: ', labels_batch.shape)
        print('labels_batch: ', labels_batch)

        #forward pass
        outputs = model(input_ids_batch, labels=labels_batch)
        accuracy_batch = accuracy(outputs.logits, labels_batch)
        avg_training_acc += accuracy_batch.item()

        loss = outputs.loss
        avg_training_loss += loss.item()
        #if t ==0:
        #    print("loss at step ", t, " : ", loss.item())
        loss.backward()
        optimizer.step()

    avg_training_loss *= 1/int(input_ids.shape[0]/N)
    avg_training_acc *= 1/int(input_ids.shape[0]/N)
    accuracy_dictionary['training_loss'].append(avg_training_loss)
    accuracy_dictionary['training'].append(avg_training_acc)
    #Print & Evaluate
    #evaluate
    model.eval()
    avg_val_seen_loss = 0.0
    avg_val_seen_acc = 0.0
    with torch.no_grad():
        for batch in val_seen_loader:
            val_seen_input_ids_batch, val_seen_labels_batch = batch

            outputs_val_seen = model(val_seen_input_ids_batch, labels=val_seen_labels_batch)
            val_accuracy_batch = accuracy(outputs_val_seen.logits, val_seen_labels_batch)
            avg_val_seen_acc += val_accuracy_batch.item()

            val_seen_loss = outputs_val_seen.loss
            avg_val_seen_loss += val_seen_loss.item()

        avg_val_seen_acc *= 1/len(val_seen_loader)
        avg_val_seen_loss *= 1/len(val_seen_loader)
        accuracy_dictionary['val_seen_acc'].append(avg_val_seen_acc)
        accuracy_dictionary['val_seen_loss'].append(avg_val_seen_loss)

input_ids shape:  torch.Size([21025, 41])
input_ids_batch shape:  torch.Size([32, 41])
labels_batch shape:  torch.Size([32, 48])
labels_batch:  tensor([[2491,  834, 6137,  ..., -100, -100, -100],
        [2491,  834, 6137,  ..., -100, -100, -100],
        [2491,  834, 6137,  ..., -100, -100, -100],
        ...,
        [2491,  834, 6137,  ..., -100, -100, -100],
        [2491,  834, 6137,  ..., -100, -100, -100],
        [2491,  834, 6137,  ..., -100, -100, -100]], device='cuda:0')


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 3.81 GiB total capacity; 2.85 GiB already allocated; 11.12 MiB free; 2.98 GiB reserved in total by PyTorch)